# Deploying Quantization Aware Trained models in INT8 using TRTorch

## Overview

Quantization Aware training (QAT) simulates quantization during training by quantizing weights and activation layers. This will help to reduce the loss in accuracy when we convert the network trained in FP32 to INT8 for faster inference. QAT introduces additional nodes in the graph which will be used to learn the dynamic ranges of weights and activation layers. In this notebook, we illustrate the following steps from training to inference of a QAT model in TRTorch.

1. [Requirements](#1)
2. [VGG16 Overview](#2)
3. [Training a baseline VGG16 model](#3)
4. [Apply Quantization](#4)
5. [Model calibration](#5)
6. [Quantization Aware training](#6)
7. [Export to Torchscript](#7)
8. [Inference using TRTorch](#8)
8. [References](#8)

<a id="1"></a>
##  1. Requirements
Please install the <a href="https://github.com/NVIDIA/TRTorch/tree/master/examples/int8/training/vgg16#prequisites">required dependencies</a> and import these libraries accordingly

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import trtorch

from torch.utils.tensorboard import SummaryWriter

from pytorch_quantization import nn as quant_nn
from pytorch_quantization import quant_modules
from pytorch_quantization.tensor_quant import QuantDescriptor
from pytorch_quantization import calib
from tqdm import tqdm

import os
import sys
sys.path.insert(0, "../examples/int8/training/vgg16")
from vgg16 import vgg16


E0831 15:09:13.151450 140586428176192 amp_wrapper.py:31] AMP is not avaialble.


<a id="2"></a>
##  2. VGG16 Overview
### Very Deep Convolutional Networks for Large-Scale Image Recognition
VGG is one of the earliest family of image classification networks that first used small (3x3) convolution filters and achieved significant improvements on ImageNet recognition challenge. The network architecture looks as follows
<img src="https://neurohive.io/wp-content/uploads/2018/11/vgg16-1-e1542731207177.png">
 

<a id="3"></a>
##  3. Training a baseline VGG16 model
We train VGG16 on CIFAR10 dataset. Define training and testing datasets and dataloaders. This will download the CIFAR 10 data in your `data` directory. Data preprocessing is performed using `torchvision` transforms. 

In [3]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# ========== Define Training dataset and dataloaders =============#
training_dataset = datasets.CIFAR10(root='./data',
                                        train=True,
                                        download=True,
                                        transform=transforms.Compose([
                                            transforms.RandomCrop(32, padding=4),
                                            transforms.RandomHorizontalFlip(),
                                            transforms.ToTensor(),
                                            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
                                        ]))

training_dataloader = torch.utils.data.DataLoader(training_dataset,
                                                      batch_size=32,
                                                      shuffle=True,
                                                      num_workers=2)

# ========== Define Testing dataset and dataloaders =============#
testing_dataset = datasets.CIFAR10(root='./data',
                                   train=False,
                                   download=True,
                                   transform=transforms.Compose([
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
                                   ]))

testing_dataloader = torch.utils.data.DataLoader(testing_dataset,
                                                 batch_size=16,
                                                 shuffle=False,
                                                 num_workers=2)


Files already downloaded and verified
Files already downloaded and verified


In [4]:
def train(model, dataloader, crit, opt, epoch):
#     global writer
    model.train()
    running_loss = 0.0
    for batch, (data, labels) in enumerate(dataloader):
        data, labels = data.cuda(), labels.cuda(non_blocking=True)
        opt.zero_grad()
        out = model(data)
        loss = crit(out, labels)
        loss.backward()
        opt.step()

        running_loss += loss.item()
        if batch % 500 == 499:
            print("Batch: [%5d | %5d] loss: %.3f" % (batch + 1, len(dataloader), running_loss / 100))
            running_loss = 0.0
        
def test(model, dataloader, crit, epoch):
    global writer
    global classes
    total = 0
    correct = 0
    loss = 0.0
    class_probs = []
    class_preds = []
    model.eval()
    with torch.no_grad():
        for data, labels in dataloader:
            data, labels = data.cuda(), labels.cuda(non_blocking=True)
            out = model(data)
            loss += crit(out, labels)
            preds = torch.max(out, 1)[1]
            class_probs.append([F.softmax(i, dim=0) for i in out])
            class_preds.append(preds)
            total += labels.size(0)
            correct += (preds == labels).sum().item()

    test_probs = torch.cat([torch.stack(batch) for batch in class_probs])
    test_preds = torch.cat(class_preds)

    return loss / total, correct / total

def save_checkpoint(state, ckpt_path="checkpoint.pth"):
    torch.save(state, ckpt_path)
    print("Checkpoint saved")

*Define the VGG model that we are going to perfom QAT on.*

In [5]:
# CIFAR 10 has 10 classes
model = vgg16(num_classes=len(classes), init_weights=False)
model = model.cuda()

In [6]:
# Declare Learning rate
lr = 0.1
state = {}
state["lr"] = lr

# Use cross entropy loss for classification and SGD optimizer
crit = nn.CrossEntropyLoss()
opt = optim.SGD(model.parameters(), lr=state["lr"], momentum=0.9, weight_decay=1e-4)


# Adjust learning rate based on epoch number
def adjust_lr(optimizer, epoch):
    global state
    new_lr = lr * (0.5**(epoch // 12)) if state["lr"] > 1e-7 else state["lr"]
    if new_lr != state["lr"]:
        state["lr"] = new_lr
        print("Updating learning rate: {}".format(state["lr"]))
        for param_group in optimizer.param_groups:
            param_group["lr"] = state["lr"]

In [7]:
# Train the model for 25 epochs to get ~80% accuracy.
num_epochs=25
for epoch in range(num_epochs):
    adjust_lr(opt, epoch)
    print('Epoch: [%5d / %5d] LR: %f' % (epoch + 1, num_epochs, state["lr"]))

    train(model, training_dataloader, crit, opt, epoch)
    test_loss, test_acc = test(model, testing_dataloader, crit, epoch)

    print("Test Loss: {:.5f} Test Acc: {:.2f}%".format(test_loss, 100 * test_acc))
    
save_checkpoint({'epoch': epoch + 1,
                 'model_state_dict': model.state_dict(),
                 'acc': test_acc,
                 'opt_state_dict': opt.state_dict(),
                 'state': state},
                ckpt_path="vgg16_base_ckpt")

Epoch: [    1 /    25] LR: 0.100000


/home/dperi/Downloads/py3/lib/python3.6/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Batch: [  500 |  1563] loss: 12.466
Batch: [ 1000 |  1563] loss: 10.726
Batch: [ 1500 |  1563] loss: 10.289
Test Loss: 0.12190 Test Acc: 19.86%
Epoch: [    2 /    25] LR: 0.100000
Batch: [  500 |  1563] loss: 10.107
Batch: [ 1000 |  1563] loss: 9.986
Batch: [ 1500 |  1563] loss: 9.994
Test Loss: 0.12230 Test Acc: 21.54%
Epoch: [    3 /    25] LR: 0.100000
Batch: [  500 |  1563] loss: 9.826
Batch: [ 1000 |  1563] loss: 9.904
Batch: [ 1500 |  1563] loss: 9.771
Test Loss: 0.11709 Test Acc: 22.71%
Epoch: [    4 /    25] LR: 0.100000
Batch: [  500 |  1563] loss: 9.760
Batch: [ 1000 |  1563] loss: 9.629
Batch: [ 1500 |  1563] loss: 9.642
Test Loss: 0.11945 Test Acc: 23.89%
Epoch: [    5 /    25] LR: 0.100000
Batch: [  500 |  1563] loss: 9.590
Batch: [ 1000 |  1563] loss: 9.489
Batch: [ 1500 |  1563] loss: 9.468
Test Loss: 0.11180 Test Acc: 30.01%
Epoch: [    6 /    25] LR: 0.100000
Batch: [  500 |  1563] loss: 9.281
Batch: [ 1000 |  1563] loss: 9.057
Batch: [ 1500 |  1563] loss: 8.957
Test L

<a id="4"></a>
##  4. Apply Quantization

`quant_modules.initialize()` will ensure quantized version of modules will be called instead of original modules. For example, when you define a model with convolution, linear, pooling layers, `QuantConv2d`, `QuantLinear` and `QuantPooling` will be called. `QuantConv2d` basically wraps quantizer nodes around inputs and weights of regular `Conv2d`. Please refer to all the <a href="https://github.com/NVIDIA/TensorRT/tree/master/tools/pytorch-quantization/pytorch_quantization/nn/modules">quantized modules</a> in pytorch-quantization toolkit for more information. A `QuantConv2d` is represented in `pytorch-quantization` toolkit as follows.

```
def forward(self, input):
        # the actual quantization happens in the next level of the class hierarchy
        quant_input, quant_weight = self._quant(input)

        if self.padding_mode == 'circular':
            expanded_padding = ((self.padding[1] + 1) // 2, self.padding[1] // 2,
                                (self.padding[0] + 1) // 2, self.padding[0] // 2)
            output = F.conv2d(F.pad(quant_input, expanded_padding, mode='circular'),
                              quant_weight, self.bias, self.stride,
                              _pair(0), self.dilation, self.groups)
        else:
            output = F.conv2d(quant_input, quant_weight, self.bias, self.stride, self.padding, self.dilation,
                              self.groups)

        return output
```

In [8]:
quant_modules.initialize()

In [9]:
# All the regular conv, FC layers will be converted to their quantozed counterparts due to quant_modules.initialize()
qat_model = vgg16(num_classes=len(classes), init_weights=False)
qat_model = qat_model.cuda()

In [10]:
# vgg16_base_ckpt is the checkpoint generated from Step 3 : Training a baseline VGG16 model.
ckpt = torch.load("./vgg16_base_ckpt")
modified_state_dict={}
for key, val in ckpt["model_state_dict"].items():
    # Remove 'module.' from the key names
    if key.startswith('module'):
        modified_state_dict[key[7:]] = val
    else:
        modified_state_dict[key] = val

# Load the pre-trained checkpoint
qat_model.load_state_dict(modified_state_dict)
opt.load_state_dict(ckpt["opt_state_dict"])

<a id="5"></a>
##  5. Model Calibration

The quantizer nodes introduced in the model around desired layers capture the dynamic range (min_value, max_value) that is observed by the layer. Calibration is the process of computing the dynamic range of these layers by passing calibration data, which is usually a subset of training or validation data. There are different ways of calibration: `max`, `histogram` and `entropy`. We use `max` calibration technique as it is simple and effective. 

In [11]:
def compute_amax(model, **kwargs):
    # Load calib result
    for name, module in model.named_modules():
        if isinstance(module, quant_nn.TensorQuantizer):
            if module._calibrator is not None:
                if isinstance(module._calibrator, calib.MaxCalibrator):
                    module.load_calib_amax()
                else:
                    module.load_calib_amax(**kwargs)
            print(F"{name:40}: {module}")
    model.cuda()

def collect_stats(model, data_loader, num_batches):
    """Feed data to the network and collect statistics"""
    # Enable calibrators
    for name, module in model.named_modules():
        if isinstance(module, quant_nn.TensorQuantizer):
            if module._calibrator is not None:
                module.disable_quant()
                module.enable_calib()
            else:
                module.disable()

    # Feed data to the network for collecting stats
    for i, (image, _) in tqdm(enumerate(data_loader), total=num_batches):
        model(image.cuda())
        if i >= num_batches:
            break

    # Disable calibrators
    for name, module in model.named_modules():
        if isinstance(module, quant_nn.TensorQuantizer):
            if module._calibrator is not None:
                module.enable_quant()
                module.disable_calib()
            else:
                module.enable()

def calibrate_model(model, model_name, data_loader, num_calib_batch, calibrator, hist_percentile, out_dir):
    """
        Feed data to the network and calibrate.
        Arguments:
            model: classification model
            model_name: name to use when creating state files
            data_loader: calibration data set
            num_calib_batch: amount of calibration passes to perform
            calibrator: type of calibration to use (max/histogram)
            hist_percentile: percentiles to be used for historgram calibration
            out_dir: dir to save state files in
    """

    if num_calib_batch > 0:
        print("Calibrating model")
        with torch.no_grad():
            collect_stats(model, data_loader, num_calib_batch)

        if not calibrator == "histogram":
            compute_amax(model, method="max")
            calib_output = os.path.join(
                out_dir,
                F"{model_name}-max-{num_calib_batch*data_loader.batch_size}.pth")
            torch.save(model.state_dict(), calib_output)
        else:
            for percentile in hist_percentile:
                print(F"{percentile} percentile calibration")
                compute_amax(model, method="percentile")
                calib_output = os.path.join(
                    out_dir,
                    F"{model_name}-percentile-{percentile}-{num_calib_batch*data_loader.batch_size}.pth")
                torch.save(model.state_dict(), calib_output)

            for method in ["mse", "entropy"]:
                print(F"{method} calibration")
                compute_amax(model, method=method)
                calib_output = os.path.join(
                    out_dir,
                    F"{model_name}-{method}-{num_calib_batch*data_loader.batch_size}.pth")
                torch.save(model.state_dict(), calib_output)

In [12]:
#Calibrate the model using max calibration technique.
with torch.no_grad():
    calibrate_model(
        model=qat_model,
        model_name="vgg16",
        data_loader=training_dataloader,
        num_calib_batch=32,
        calibrator="max",
        hist_percentile=[99.9, 99.99, 99.999, 99.9999],
        out_dir="./")

Calibrating model


100%|██████████| 32/32 [00:00<00:00, 85.05it/s]
W0831 15:32:46.956144 140586428176192 tensor_quantizer.py:173] Disable MaxCalibrator
W0831 15:32:46.957227 140586428176192 tensor_quantizer.py:173] Disable MaxCalibrator
W0831 15:32:46.958076 140586428176192 tensor_quantizer.py:173] Disable MaxCalibrator
W0831 15:32:46.958884 140586428176192 tensor_quantizer.py:173] Disable MaxCalibrator
W0831 15:32:46.959697 140586428176192 tensor_quantizer.py:173] Disable MaxCalibrator
W0831 15:32:46.960512 140586428176192 tensor_quantizer.py:173] Disable MaxCalibrator
W0831 15:32:46.961301 140586428176192 tensor_quantizer.py:173] Disable MaxCalibrator
W0831 15:32:46.962079 140586428176192 tensor_quantizer.py:173] Disable MaxCalibrator
W0831 15:32:46.962872 140586428176192 tensor_quantizer.py:173] Disable MaxCalibrator
W0831 15:32:46.963665 140586428176192 tensor_quantizer.py:173] Disable MaxCalibrator
W0831 15:32:46.964508 140586428176192 tensor_quantizer.py:173] Disable MaxCalibrator
W0831 15:32:46.96

features.0._input_quantizer             : TensorQuantizer(8bit narrow fake per-tensor amax=2.7537 calibrator=MaxCalibrator scale=1.0 quant)
features.0._weight_quantizer            : TensorQuantizer(8bit narrow fake axis=0 amax=[0.0287, 4.4272](64) calibrator=MaxCalibrator scale=1.0 quant)
features.3._input_quantizer             : TensorQuantizer(8bit narrow fake per-tensor amax=30.1997 calibrator=MaxCalibrator scale=1.0 quant)


W0831 15:32:57.006651 140586428176192 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0831 15:32:57.009306 140586428176192 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([128, 1, 1, 1]).
W0831 15:32:57.011739 140586428176192 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0831 15:32:57.014180 140586428176192 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([128, 1, 1, 1]).
W0831 15:32:57.016433 140586428176192 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0831 15:32:57.018157 140586428176192 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([256, 1, 1, 1]).
W0831 15:32:57.019830 140586428176192 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0831 15:32:57.021619 140586428176192 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([256, 1, 1, 1]).
W0831 15:32:57.023381 140586428176192 tensor_quantizer.py:237] Load calibrated amax, shape=torch

features.3._weight_quantizer            : TensorQuantizer(8bit narrow fake axis=0 amax=[0.0137, 2.2506](64) calibrator=MaxCalibrator scale=1.0 quant)
features.7._input_quantizer             : TensorQuantizer(8bit narrow fake per-tensor amax=16.2026 calibrator=MaxCalibrator scale=1.0 quant)
features.7._weight_quantizer            : TensorQuantizer(8bit narrow fake axis=0 amax=[0.0602, 1.3986](128) calibrator=MaxCalibrator scale=1.0 quant)
features.10._input_quantizer            : TensorQuantizer(8bit narrow fake per-tensor amax=9.1012 calibrator=MaxCalibrator scale=1.0 quant)
features.10._weight_quantizer           : TensorQuantizer(8bit narrow fake axis=0 amax=[0.0841, 0.9074](128) calibrator=MaxCalibrator scale=1.0 quant)
features.14._input_quantizer            : TensorQuantizer(8bit narrow fake per-tensor amax=10.0201 calibrator=MaxCalibrator scale=1.0 quant)
features.14._weight_quantizer           : TensorQuantizer(8bit narrow fake axis=0 amax=[0.0921, 0.7349](256) calibrator=MaxCal

<a id="6"></a>
##  6. Quantization Aware Training

In this phase, we finetune the model weights and leave the quantizer node values frozen. The dynamic ranges for each layer obtained from the calibration are kept constant while the weights of the model are finetuned to be close to the accuracy of original FP32 model (model without quantizer nodes) is preserved. Usually the finetuning of QAT model should be quick compared to the full training of the original model. Use QAT to fine-tune for around 10% of the original training schedule with an annealing learning-rate. Please refer to <a href="https://developer.nvidia.com/blog/achieving-fp32-accuracy-for-int8-inference-using-quantization-aware-training-with-tensorrt/">Achieving FP32 Accuracy for INT8 Inference Using Quantization Aware Training with NVIDIA TensorRT</a> for detailed recommendations. For this VGG model, it is enough to finetune for 1 epoch to get acceptable accuracy. 
During finetuning with QAT, the quantization is applied as a composition of `max`, `clamp`, `round` and `mul` ops. 
```
# amax is absolute maximum value for an input
# The upper bound for integer quantization (127 for int8)
max_bound = torch.tensor((2.0**(num_bits - 1 + int(unsigned))) - 1.0, device=amax.device)
scale = max_bound / amax
outputs = torch.clamp((inputs * scale).round_(), min_bound, max_bound)
```
<a href="https://github.com/NVIDIA/TensorRT/blob/8.0.1/tools/pytorch-quantization/pytorch_quantization/tensor_quant.py">tensor_quant function</a> in `pytorch_quantization` toolkit is responsible for the above tensor quantization. Usually, per channel quantization is recommended for weights, while per tensor quantization is recommended for activations in a network.
During inference, we use `torch.fake_quantize_per_tensor_affine` and `torch.fake_quantize_per_channel_affine` to perform quantization as this is easier to convert into corresponding TensorRT operators. Please refer to next sections for more details on how these operators are exported in torchscript and converted in TRTorch.

In [13]:
# Finetune the QAT model for 1 epoch
num_epochs=1
for epoch in range(num_epochs):
    adjust_lr(opt, epoch)
    print('Epoch: [%5d / %5d] LR: %f' % (epoch + 1, num_epochs, state["lr"]))

    train(qat_model, training_dataloader, crit, opt, epoch)
    test_loss, test_acc = test(qat_model, testing_dataloader, crit, epoch)

    print("Test Loss: {:.5f} Test Acc: {:.2f}%".format(test_loss, 100 * test_acc))
    
save_checkpoint({'epoch': epoch + 1,
                 'model_state_dict': qat_model.state_dict(),
                 'acc': test_acc,
                 'opt_state_dict': opt.state_dict(),
                 'state': state},
                ckpt_path="vgg16_qat_ckpt")

Updating learning rate: 0.1
Epoch: [    1 /     1] LR: 0.100000
Batch: [  500 |  1563] loss: 2.694
Batch: [ 1000 |  1563] loss: 2.682
Batch: [ 1500 |  1563] loss: 2.624
Test Loss: 0.03277 Test Acc: 83.58%
Checkpoint saved


<a id="7"></a>
##  7. Export to Torchscript
Export the model to Torch script. Trace the model and convert it into torchscript for deployment. To learn more about Torchscript, please refer to https://pytorch.org/docs/stable/jit.html. Setting `quant_nn.TensorQuantizer.use_fb_fake_quant = True` enables the QAT model to use `torch.fake_quantize_per_tensor_affine` and `torch.fake_quantize_per_channel_affine` operators instead of `tensor_quant` function to export quantization operators. In torchscript, they are represented as `aten::fake_quantize_per_tensor_affine` and `aten::fake_quantize_per_channel_affine`. 

In [16]:
quant_nn.TensorQuantizer.use_fb_fake_quant = True
with torch.no_grad():
    data = iter(testing_dataloader)
    images, _ = data.next()
    jit_model = torch.jit.trace(qat_model, images.to("cuda"))
    torch.jit.save(jit_model, "trained_vgg16_qat.jit.pt")

E0831 15:41:34.662368 140586428176192 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0831 15:41:34.664751 140586428176192 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0831 15:41:34.671072 140586428176192 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0831 15:41:34.671867 140586428176192 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0831 15:41:34.683352 140586428176192 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0831 15:41:34.684193 140586428176192 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0831 15:41:34.687814 140586428176192 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0831 15:41:34.688531 140586428176192 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0831 15:41:34.698150 140586428176192 tensor_quantizer.py:120] Fake quantize mode doesn't use scale expl

E0831 15:41:34.962537 140586428176192 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0831 15:41:34.963227 140586428176192 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0831 15:41:34.970601 140586428176192 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0831 15:41:34.971469 140586428176192 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0831 15:41:34.974947 140586428176192 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0831 15:41:34.975533 140586428176192 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0831 15:41:34.985072 140586428176192 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0831 15:41:34.985844 140586428176192 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0831 15:41:34.988213 140586428176192 tensor_quantizer.py:120] Fake quantize mode doesn't use scale expl

<a id="8"></a>
##  8. Inference using TRTorch
In this phase, we run the exported torchscript graph of VGG QAT using TRTorch. TRTorch is a Pytorch-TensorRT compiler which converts Torchscript graphs into TensorRT. TensorRT 8.0 supports inference of quantization aware trained models and introduces new APIs; `QuantizeLayer` and `DequantizeLayer`. We can observe the entire VGG QAT graph quantization nodes from the debug log of TRTorch. To enable debug logging, you can set `trtorch.logging.set_reportable_log_level(trtorch.logging.Level.Debug)`. For example, `QuantConv2d` layer from `pytorch_quantization` toolkit is represented as follows in Torchscript
```
%quant_input : Tensor = aten::fake_quantize_per_tensor_affine(%x, %636, %637, %638, %639)
%quant_weight : Tensor = aten::fake_quantize_per_channel_affine(%394, %640, %641, %637, %638, %639)
%input.2 : Tensor = aten::_convolution(%quant_input, %quant_weight, %395, %687, %688, %689, %643, %690, %642, %643, %643, %644, %644)
```
`aten::fake_quantize_per_*_affine` is converted into `QuantizeLayer` + `DequantizeLayer` in TRTorch internally. Please refer to <a href="https://github.com/NVIDIA/TRTorch/blob/master/core/conversion/converters/impl/quantization.cpp">quantization op converters</a> in TRTorch.

In [17]:
qat_model = torch.jit.load("trained_vgg16_qat.jit.pt").eval()

compile_spec = {"inputs": [trtorch.Input([16, 3, 32, 32])],
                "op_precision": torch.int8,
                }
trt_mod = trtorch.compile(qat_model, compile_spec)

test_loss, test_acc = test(trt_mod, testing_dataloader, crit, 0)
print("VGG QAT accuracy using TensorRT: {:.2f}%".format(100 * test_acc))

VGG QAT accuracy using TensorRT: 83.59%


<a id="9"></a>
##  9. References
* <a href="https://arxiv.org/pdf/1409.1556.pdf">Very Deep Convolution Networks for large scale Image Recognition</a>
* <a href="https://developer.nvidia.com/blog/achieving-fp32-accuracy-for-int8-inference-using-quantization-aware-training-with-tensorrt/">Achieving FP32 Accuracy for INT8 Inference Using Quantization Aware Training with NVIDIA TensorRT</a>
* <a href="https://github.com/NVIDIA/TRTorch/tree/master/examples/int8/training/vgg16#quantization-aware-fine-tuning-for-trying-out-qat-workflows">QAT workflow for VGG16</a>
* <a href="https://github.com/NVIDIA/TRTorch/tree/master/examples/int8/qat">Deploying VGG QAT model in C++ using TRTorch</a>
* <a href="https://github.com/NVIDIA/TensorRT/tree/master/tools/pytorch-quantization">Pytorch-quantization toolkit from NVIDIA</a>
* <a href="https://docs.nvidia.com/deeplearning/tensorrt/pytorch-quantization-toolkit/docs/userguide.html">Pytorch quantization toolkit userguide</a>
* <a href="https://arxiv.org/pdf/2004.09602.pdf">Quantization basics</a>